# How to plot infection rates by contact models

The following simulation is identical to the simulation in the [tutorial on contact models](how_to_specify_contact_models.ipynb). The heatmap shows the contribution of contact models to new infections for each day in the simulation. The values are shares in relation to the whole population.

In [1]:
import warnings

import holoviews as hv
import numpy as np
import pandas as pd

import sid
from sid.config import INDEX_NAMES
from sid.plotting import plot_infection_rates_by_contact_models

warnings.filterwarnings(
    "ignore", message="indexing past lexsort depth may impact performance."
)

In [2]:
n_individuals = 10_000
available_ages = [
    "0-9",
    "10-19",
    "20-29",
    "30-39",
    "40-49",
    "50-59",
    "60-69",
    "70-79",
    "80-100",
]

ages = np.random.choice(available_ages, size=n_individuals)
regions = np.random.choice(["North", "East", "South", "West"], size=n_individuals)
hh_id = pd.Series(np.random.choice(int(n_individuals / 1.6), size=n_individuals))

initial_states = pd.DataFrame(
    {"age_group": ages, "region": regions, "hh_id": hh_id}
).astype("category")
initial_states.head(5)

,age_group,region,hh_id
0,40-49,North,4479
1,20-29,North,2948
2,30-39,West,134
3,80-100,South,3999
4,20-29,West,2222


In [3]:
contact_models = {}


def random_encounters(states, params, seed):
    np.random.seed(seed)
    contacts = np.random.choice(np.arange(3), size=states.shape[0])
    return pd.Series(index=states.index, data=contacts)


contact_models["random_encounters"] = {
    "model": random_encounters,
    "assort_by": ["age_group", "region"],
    "is_recurrent": False,
}


def meet_household(states, params, seed):
    return pd.Series(index=states.index, data=True)


contact_models["meet_household"] = {
    "model": meet_household,
    "assort_by": "hh_id",
    "is_recurrent": True,
}

In [4]:
params = sid.load_epidemiological_parameters()
immunity_params = pd.read_csv("immunity_params.csv", index_col=INDEX_NAMES)

params = pd.concat((params, immunity_params))

params.loc[("assortative_matching", "random_encounters", "age_group"), "value"] = 0.2
params.loc[("assortative_matching", "random_encounters", "region"), "value"] = 0.9
params.loc[("infection_prob", "random_encounters", "random_encounters"), "value"] = 0.1
params.loc[("infection_prob", "meet_household", "meet_household"), "value"] = 0.15

In [9]:
simulate = sid.get_simulate_func(
    params=params,
    initial_states=initial_states,
    contact_models=contact_models,
    duration={"start": "2020-03-01", "end": "2020-04-01"},
    saved_columns={"other": ["channel_infected_by_contact"]},
    seed=144,
)

result = simulate(params)

Start the simulation...
2020-04-01: 100%|██████████| 32/32 [00:03<00:00, 10.35it/s]


In [11]:
contact_models

value  \
category             subcategory        name                                       
health_system        icu_limit_relative icu_limit_relative               50.0000   
cd_infectious_true   all                1                                 0.3900   
                                        2                                 0.3500   
                                        3                                 0.2200   
                                        5                                 0.0400   
...                                                                          ...   
immunity             immunity_waning    slope_after_maximum_vaccination  -0.0002   
assortative_matching random_encounters  age_group                         0.2000   
                                        region                            0.9000   
infection_prob       random_encounters  random_encounters                 0.1000   
                     meet_household     meet_household                    0.1500   

                                                                         note  \
category             subcategory        name                                    
health_system        icu_limit_relative icu_limit_relative                NaN   
cd_infectious_true   all                1                                 NaN   
                                        2                                 NaN   
                                        3                                 NaN   
                                        5                                 NaN   
...                                                                       ...   
immunity             immunity_waning    slope_after_maximum_vaccination   NaN   
assortative_matching random_encounters  age_group                         NaN   
                                        region                            NaN   
infection_prob       random_encounters  random_encounters                 NaN   
                     meet_household     meet_household                    NaN   

                                                                         source  
category             subcategory        name                                     
health_system        icu_limit_relative icu_limit_relative                  NaN  
cd_infectious_true   all                1                                   NaN  
                                        2                                   NaN  
                                        3                                   NaN  
                                        5                                   NaN  
...                                                                         ...  
immunity             immunity_waning    slope_after_maximum_vaccination     NaN  
assortative_matching random_encounters  age_group                           NaN  
                                        region                              NaN  
infection_prob       random_encounters  random_encounters                   NaN  
                     meet_household     meet_household                      NaN  

[139 rows x 3 columns]

In [7]:
result["time_series"]

,age_group,newly_deceased,newly_infected,ever_vaccinated,date,hh_id,needs_icu,channel_infected_by_contact,symptomatic,region,ever_infected,new_known_case,cd_infectious_false,dead,newly_vaccinated,is_tested_positive_by_rapid_test,knows_immune,knows_infectious,infectious
npartitions=48,,,,,,,,,,,,,,,,,,,
,category[unknown],bool,bool,bool,datetime64[ns],category[unknown],bool,category[unknown],bool,category[unknown],bool,bool,int16,float16,bool,bool,bool,bool,bool
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
heatmap = plot_infection_rates_by_contact_models(result["time_series"], unit="share")
heatmap

TypeError: Attempt to read as category a field that was not stored as such